In [2]:
from fastai.text import *
import pandas as pd

In [ ]:
#data_path = Config.data_path()


In [3]:
path = Path('../data/processed')

In [4]:
df = pd.read_csv(path/'03_classification_valid_train.csv', error_bad_lines=False, index_col=0)

In [5]:
df

,Primary Category,Secondary Category,Title,URL,scheme,netloc,url1,url2,url3,url4,url5,url6,is_valid
0,Arts & Humanities,Architecture,68 Dean Street,http://www.sixty8.com/,http,"['www', 'sixty8', 'com']",www,sixty8,com,NaN,NaN,NaN,False
1,Arts & Humanities,Architecture,Abandoned Communities,http://www.abandonedcommunities.co.uk/,http,"['www', 'abandonedcommunities', 'co', 'uk']",www,abandonedcommunities,co,uk,NaN,NaN,False
2,Arts & Humanities,Architecture,Alexander Thomson Society,http://www.greekthomson.com/,http,"['www', 'greekthomson', 'com']",www,greekthomson,com,NaN,NaN,NaN,False
3,Arts & Humanities,Architecture,"Arab British Centre, The",http://www.arabbritishcentre.org.uk/,http,"['www', 'arabbritishcentre', 'org', 'uk']",www,arabbritishcentre,org,uk,NaN,NaN,False
4,Arts & Humanities,Architecture,Architectural Association School of Architecture,http://www.aaschool.ac.uk/,http,"['www', 'aaschool', 'ac', 'uk']",www,aaschool,ac,uk,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17043,History,Slavery and Abolition in the Caribbean,Westminster City Council: Westminster and the ...,http://www.westminster.gov.uk/workspace/assets...,http,"['www', 'westminster', 'gov', 'uk']",www,westminster,gov,uk,NaN,NaN,True
20116,Science & Technology,Cambridge Network,Aveillant Ltd,http://www.aveillant.com/,http,"['www', 'aveillant', 'com']",www,aveillant,com,NaN,NaN,NaN,True
19564,"Politics, Political Theory and Political Systems","Religion, politics and law since 2005",Religion Law UK,http://www.neiladdison.pwp.blueyonder.co.uk/re...,http,"['www', 'neiladdison', 'pwp', 'blueyonder', 'c...",www,neiladdison,pwp,blueyonder,co,uk,True
25551,Society & Culture,Travel & Tourism,Backpacking Guide,http://www.the-backpacking-site.com/,http,"['www', 'the-backpacking-site', 'com']",www,the-backpacking-site,com,NaN,NaN,NaN,True


In [118]:
df.Title.to_csv(path/'lm_data.txt', header=None, index=None, sep=' ', mode='w')

In [124]:
for i,k in df.iterrows():
    txt = k['Title']
    with open(f'../data/processed/text/{i}.txt','a') as f:
        f.write(txt)

In [ ]:
###data = Path('/Users/dvanstrien/Desktop/out_mul/0002084/1857/0608/')

data = Path('/Users/dvanstrien/Desktop/out_mul/0002084/1857/')

In [138]:
data = Path('../data/processed/')

data.ls()

[PosixPath('../data/processed/Untitled Folder'),
 PosixPath('../data/processed/.gitkeep'),
 PosixPath('../data/processed/models'),
 PosixPath('../data/processed/lm_data.txt'),
 PosixPath('../data/processed/03_classification_valid_train.csv'),
 PosixPath('../data/processed/.ipynb_checkpoints'),
 PosixPath('../data/processed/text'),
 PosixPath('../data/processed/tmp')]

## add line for download SPP data

In [140]:
dlm = (TextList.from_folder(data, extensions='.txt',processor=[OpenFileProcessor(),SPProcessor(model_type='unigram')])
       .split_by_rand_pct().
       label_for_lm().databunch(bs=32))

In [ ]:
dlm

## LM

# Use title to start

In [175]:
LM_learn = language_model_learner(dlm, AWD_LSTM, pretrained=True, drop_mult=0.5)

In [ ]:
LM_learn.lr_find()

In [ ]:
LM_learn.recorder.plot()

In [176]:
LM_learn.fit_one_cycle(5,max_lr=1e-01)

epoch,train_loss,valid_loss,accuracy,time
0,3.938918,3.893412,0.398036,21:50
1,3.860417,3.748144,0.412640,22:14
2,3.534910,3.337014,0.450548,21:15
3,3.030882,2.886694,0.502041,21:21
4,2.712003,2.749291,0.521237,21:28


In [177]:
#LM_learn.path
LM_learn.save_encoder('ft_enc')

In [182]:
c

PosixPath('../data/processed')

LM_learnassification 

In [178]:
model = Path('../data/processed/')

In [179]:
data_class = (TextList.
        from_df(df,path='.',cols=['Title'],processor=SPProcessor.load(model))
        .split_from_df('is_valid')
        .label_from_df('Primary Category')
        .databunch(bs=32))

In [180]:
learn_class = text_classifier_learner(data_class,AWD_LSTM, drop_mult=0.5)

In [187]:
learn_class.path = LM_learn.path


In [189]:
learn_class.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (26798 items)
x: TextList
▁xxbos ▁68 ▁xxmaj ▁dean ▁xxmaj ▁street,▁xxbos ▁xxmaj ▁abandoned ▁xxmaj ▁communities,▁xxbos ▁xxmaj ▁alexander ▁xxmaj ▁thomson ▁xxmaj ▁society,▁xxbos ▁xxmaj ▁arab ▁xxmaj ▁british ▁xxmaj ▁centre , ▁xxmaj ▁the,▁xxbos ▁xxmaj ▁architectural ▁xxmaj ▁association ▁xxmaj ▁school ▁of ▁xxmaj ▁architecture
y: CategoryList
Arts & Humanities,Arts & Humanities,Arts & Humanities,Arts & Humanities,Arts & Humanities
Path: .;

Valid: LabelList (8039 items)
x: TextList
▁xxbos ▁xxmaj ▁bio mas s ▁xxmaj ▁energy ▁xxmaj ▁centre,▁xxbos ▁xxmaj ▁business ▁and ▁xxmaj ▁professional ▁xxmaj ▁women ▁xxup ▁uk ▁xxup ▁( b p w uk ),▁xxbos ▁ spec tra : ▁xxmaj ▁submission , ▁xxmaj ▁preservation ▁and ▁xxmaj ▁exposure ▁of ▁xxmaj ▁chemistry ▁xxmaj ▁teaching ▁and ▁xxmaj ▁research ▁xxmaj ▁data,▁xxbos ▁xxmaj ▁f - word : ▁xxmaj ▁gender - test ing ▁for ▁' s us pic ious - looking ' ▁female ▁xxmaj ▁olympic ▁athletes ▁in ▁xxmaj ▁beijing,▁xxbos ▁xxmaj ▁news ▁

In [ ]:
learn_class.fit_one_cycle(20)

epoch,train_loss,valid_loss,accuracy,time
0,2.709109,2.429864,0.341336,12:23
1,2.142002,1.991313,0.358751,11:11
2,2.134703,1.964828,0.356885,11:19
3,2.115787,1.947505,0.361612,10:55


## LM on the URL 